In [1]:
import operator
import math
import pandas as pd
import numpy as np

In [82]:
ratings_expand = {
    '마동석': {
        '택시운전사': 3.5,
        '남한산성': 1.5,
        '킹스맨:골든서클': 3.0,
        '범죄도시': 3.5,
        '아이 캔 스피크': 2.5,
        '꾼': 3.0,
    },
    '이정재': {
        '택시운전사': 5.0,
        '남한산성': 4.5,
        '킹스맨:골든서클': 0.5,
        '범죄도시': 1.5,
        '아이 캔 스피크': 4.5,
        '꾼': 5.0,
    },
    '윤계상': {
        '택시운전사': 3.0,
        '남한산성': 2.5,
        '킹스맨:골든서클': 1.5,
        '범죄도시': 3.0,
        '꾼': 3.0,
        '아이 캔 스피크': 3.5,
    },
    '설경구': {
        '택시운전사': 2.5,
        '남한산성': 3.0,
        '범죄도시': 4.5,
        '꾼': 4.0,
    },
    '최홍만': {
        '남한산성': 4.5,
        '킹스맨:골든서클': 3.0,
        '꾼': 4.5,
        '범죄도시': 3.0,
        '아이 캔 스피크': 2.5,
    },
    '홍수환': {
        '택시운전사': 3.0,
        '남한산성': 4.0,
        '킹스맨:골든서클': 1.0,
        '범죄도시': 3.0,
        '꾼': 3.5,
        '아이 캔 스피크': 2.0,
    },
    '나원탁': {
        '택시운전사': 3.0,
        '남한산성': 4.0,
        '꾼': 3.0,
        '범죄도시': 5.0,
        '아이 캔 스피크': 3.5,
    },
    '소이현': {
        '남한산성': 4.5, 
        '아이 캔 스피크': 1.0,
        '범죄도시': 4.0
    }
}

df = pd.DataFrame(ratings_expand)
df

,마동석,이정재,윤계상,설경구,최홍만,홍수환,나원탁,소이현
택시운전사,3.5,5.0,3.0,2.5,NaN,3.0,3.0,NaN
남한산성,1.5,4.5,2.5,3.0,4.5,4.0,4.0,4.5
킹스맨:골든서클,3.0,0.5,1.5,NaN,3.0,1.0,NaN,NaN
범죄도시,3.5,1.5,3.0,4.5,3.0,3.0,5.0,4.0
아이 캔 스피크,2.5,4.5,3.5,NaN,2.5,2.0,3.5,1.0
꾼,3.0,5.0,3.0,4.0,4.5,3.5,3.0,NaN


In [92]:
def getStatistics(df):
    colStd = df.std()
    colMean = df.mean()
    return colStd, colMean

def getSim(df,target):
    df = df[df[target].notnull()]
    temp_key = []
    temp_value = []

    for name in df.columns:
        if name != target:
            if len(df[df[name].notnull()]) < len(df[target]):
                df2 = df[df[name].notnull()].copy()
                colStd,colMean = getStatistics(df2)
                df2[name] = df2[name] - colMean[name]
                df2[target] = df2[target] - colMean[target]

                temp_key.append(name)
                temp_value.append(df2[[target,name]].prod(axis=1).sum() / (len(df2[name]) - 1) / (colStd[target] * colStd[name]))

            else :
                # print(df[name])
                colStd,colMean = getStatistics(df)
                df[name] = df[name] - colMean[name]
                df[target] = df[target] - colMean[target]

                temp_key.append(name)
                temp_value.append(df[[target,name]].prod(axis=1).sum() / (len(df[name]) - 1) / (colStd[target] * colStd[name]))
    
    pearson = dict(zip(temp_key,temp_value))
    return pearson

def getWeightAverage(df,target,similar):
    df2 = df[df[target].isnull()].copy()
    simTrueOnly = dict(filter(lambda elem:elem[1] > 0, similar.items()))
    simName = list(simTrueOnly.keys())

    for name,sim in simTrueOnly.items():
        if sim > 0:
            df2[name] = df2[name] * sim

    df2 = df2[simName]
    df3 = df2.sum(axis=1)
    temp = [] 

    for i,name in enumerate(simName):
        for j in range(len(df2.iloc[i,:])):
            if df2.iloc[i,j] > 0:
                # print(df2.iloc[i,j])
                # print(simName[j])
                temp.append(simTrueOnly[simName[j]])

        df3[i] = df3[i] / sum(temp)
        temp = []
    return df3

def getRecommendation(df,target):
    similar = getSim(df,target)
    estimatedScoreWeightedAverage = getWeightAverage(df,target,similar)
    return estimatedScoreWeightedAverage


estimatedScores = getRecommendation(df,'소이현')

estimatedScores.to_frame()



/var/folders/_v/kggs6h5567n9f1tb2crdb8300000gn/T/ipykernel_34152/247375512.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name] = df[name] - colMean[name]
/var/folders/_v/kggs6h5567n9f1tb2crdb8300000gn/T/ipykernel_34152/247375512.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[target] = df[target] - colMean[target]
/var/folders/_v/kggs6h5567n9f1tb2crdb8300000gn/T/ipykernel_34152/247375512.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

,0
택시운전사,3.000000
킹스맨:골든서클,1.916806
꾼,3.690330
